<a href="https://colab.research.google.com/github/hieulv416c/Url_content_detect/blob/main/BERT%2BMachinelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
lehieu12_cleaned_data_url68k_path = kagglehub.dataset_download('lehieu12/cleaned-data-url68k')

print('Data source import complete.')


In [ ]:


!pip install --upgrade gensim

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


# Import the libraries
import matplotlib.pyplot as plt
import matplotlib.style
import seaborn as sns

import nltk
nltk.download('punkt_tab')

!pip install transformers tensorflow numpy pandas scikit-learn nltk
!pip install underthesea  # Nếu dữ liệu là tiếng Việt



import re
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# Load data
file_path = '/kaggle/input/cleaned-data-url68k/content_cleaned.csv'

df = pd.read_csv(file_path)
print(df.head())


df = df.dropna()



df["Content"] = df["Content"].astype(str)

!pip install catboost

import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  # Thêm chuẩn hóa
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import BertTokenizer




# 📌 Tải mô hình BERT từ Hugging Face
bert_model = TFBertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 📌 Hàm chuyển văn bản thành BERT embedding (sử dụng vector CLS)
def bert_embedding(texts, batch_size=8):
    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]

        # Token hóa và padding
        encoded_inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=50, return_tensors="tf")

        # Chạy qua mô hình BERT
        outputs = bert_model(**encoded_inputs)

        # Lấy vector CLS (token đầu tiên của last_hidden_state)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings.append(batch_embeddings)

    return np.vstack(embeddings)



# Chia tập train/test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["Content"].tolist(), df["result"].tolist(), test_size=0.2, random_state=42
)



# Chuyển đổi văn bản thành embedding với batch nhỏ
X_train = bert_embedding(train_texts, batch_size=8)
X_test = bert_embedding(test_texts, batch_size=8)
y_train = np.array(train_labels, dtype=np.int64)
y_test = np.array(test_labels, dtype=np.int64)



# 📌 Danh sách mô hình ML
models = {
    "Logistic Regression": LogisticRegression(max_iter=500, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "MLP Classifier": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    "SVM": SVC(kernel="linear", random_state=42),
}

# # 📌 Danh sách các mô hình CẦN chuẩn hóa
# models_need_scaling = ["SVM", "Logistic Regression", "KNN", "MLP Classifier"]

# # Áp dụng StandardScaler cho những mô hình cần
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)


# 🏆 Huấn luyện và đánh giá mô hình
for name, model in models.items():
    print(f"\n🚀 Đang huấn luyện mô hình: {name}...")

    # # Kiểm tra nếu mô hình cần chuẩn hóa
    # if name in models_need_scaling:
    #     model.fit(X_train_scaled, y_train)
    #     y_pred = model.predict(X_test_scaled)
    # else:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Đánh giá mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"🎯 Độ chính xác: {accuracy:.4f}")
    print("📊 Báo cáo phân loại:")
    print(classification_report(y_test, y_pred))

    # Vẽ ma trận nhầm lẫn
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Không Phishing", "Phishing"], yticklabels=["Không Phishing", "Phishing"])
    plt.xlabel("Dự đoán")
    plt.ylabel("Thực tế")
    plt.title(f"🔍 Ma trận nhầm lẫn - {name}")
    plt.show()

print("🎉 Quá trình huấn luyện và đánh giá mô hình hoàn tất!")